In [1]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2026-01-24 03:54:38--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.02s   

2026-01-24 03:54:38 (12.2 MB/s) - ‘names.txt’ saved [228145/228145]



In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
import torch

In [4]:
# mapping 
chars = sorted(set(''.join(words)))
stoi = {c:i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {i:c for c, i in stoi.items()}

In [5]:
# create counting matrix
g = torch.Generator().manual_seed(2147483647)

N = torch.zeros((27, 27, 27), dtype=float)

In [6]:
for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1][ix2][ix3] += 1

In [7]:
N[0][0]
p = N[0][0].float()
p = p / p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [8]:
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

tensor([13, 19, 14,  1,  1, 20, 20,  1, 13, 13, 13, 11, 23, 13, 19, 18, 26, 11,
         7, 19])

In [9]:
# probability
P = (N+1).float()
P = P / P.sum(2, keepdim=True)

In [10]:
for i in range(5):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
        p = P[ix1, ix2]
        ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix3])
        ix1 = ix2
        ix2 = ix3
        if ix3 == 0:
            break
    print(''.join(out))

lidy.
jona.
tarishari.
eristiftq.
jaclys.


In [11]:
log_likelihood = 0.0
n = 0

for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        prob = P[ix1][ix2][ix3] 
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
nll = -log_likelihood
print(nll/n)

tensor(2.2120)


In [12]:
xs, ys = [], []

for w in words[:1]:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]

        xs.append((ix1, ix2))
        ys.append(ix3)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [13]:
xs, ys

(tensor([[ 0,  0],
         [ 0,  5],
         [ 5, 13],
         [13, 13],
         [13,  1]]),
 tensor([ 5, 13, 13,  1,  0]))

In [14]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()

In [15]:
xenc.shape

torch.Size([5, 2, 27])

In [16]:
xenc = xenc.view(-1, 54)
xenc.shape

torch.Size([5, 54])

In [17]:
W = torch.randn((54, 27), requires_grad=True)
xenc @ W

tensor([[-1.5771,  0.6682,  0.5342, -1.1169,  0.5862, -2.5231, -4.1855,  0.2095,
          1.2955, -1.2170,  2.1445, -1.6232,  1.4934,  1.7728,  1.3671, -0.6968,
          3.3716,  2.7955,  0.4625,  0.9154, -0.9253,  0.2008,  0.2318, -1.2455,
          0.7099, -0.9343, -1.2803],
        [ 1.7295,  2.4073, -0.2564,  0.6583, -0.2403,  0.5823,  0.4470, -0.0045,
         -2.7492, -2.4543,  0.1593, -1.3893, -0.6809,  1.4144,  0.7593,  0.1225,
          1.7759,  1.4090,  1.6934,  0.1073, -2.8083,  0.5971, -1.6901, -0.8414,
          0.0113,  0.8316, -0.5631],
        [-1.6070, -0.2724,  0.3078,  1.4933, -0.8161,  0.7179,  0.7075,  0.6147,
         -1.1917, -0.6544, -1.4898, -0.7996,  1.5956, -2.2862, -0.3777, -0.8104,
          0.2720,  0.1003,  1.2408, -1.5946,  1.4002,  1.2619,  0.7886, -0.6546,
          0.3533, -0.0551,  2.2099],
        [-1.7175,  0.2074, -0.4473, -0.8020, -1.7166,  0.5614, -0.3247,  0.5802,
         -3.0790, -1.4563, -1.0138, -0.1680,  1.4244, -0.9989, -0.0129, -1.3884

In [18]:
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
probs

tensor([[2.2898e-03, 2.1622e-02, 1.8912e-02, 3.6277e-03, 1.9920e-02, 8.8910e-04,
         1.6865e-04, 1.3668e-02, 4.0490e-02, 3.2822e-03, 9.4641e-02, 2.1866e-03,
         4.9351e-02, 6.5260e-02, 4.3496e-02, 5.5222e-03, 3.2284e-01, 1.8145e-01,
         1.7603e-02, 2.7687e-02, 4.3941e-03, 1.3550e-02, 1.3977e-02, 3.1902e-03,
         2.2543e-02, 4.3547e-03, 3.0809e-03],
        [9.8973e-02, 1.9494e-01, 1.3585e-02, 3.3908e-02, 1.3805e-02, 3.1429e-02,
         2.7451e-02, 1.7477e-02, 1.1233e-03, 1.5084e-03, 2.0588e-02, 4.3758e-03,
         8.8863e-03, 7.2226e-02, 3.7514e-02, 1.9844e-02, 1.0368e-01, 7.1838e-02,
         9.5464e-02, 1.9545e-02, 1.0588e-03, 3.1895e-02, 3.2392e-03, 7.5684e-03,
         1.7755e-02, 4.0327e-02, 9.9968e-03],
        [4.1187e-03, 1.5645e-02, 2.7948e-02, 9.1450e-02, 9.0835e-03, 4.2116e-02,
         4.1681e-02, 3.7986e-02, 6.2390e-03, 1.0677e-02, 4.6309e-03, 9.2343e-03,
         1.0131e-01, 2.0882e-03, 1.4081e-02, 9.1347e-03, 2.6965e-02, 2.2710e-02,
         7.1044e-

In [19]:
W = torch.randn((54, 27), requires_grad=True)

# forward
xenc = F.one_hot(xs, num_classes=27).float()
xenc = xenc.view(-1, 54)
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(5), ys].log().mean()

In [20]:
loss

tensor(3.9745, grad_fn=<NegBackward0>)

In [21]:
W.grad = None
loss.backward()

In [22]:
W.data += -1 * W.grad

In [23]:
xs, ys = [], []
for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]

        xs.append((ix1, ix2))
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), requires_grad=True, generator=g)